# <center>Oil Properties Estimation</center>

### <center>Sept 2014<br>By: Bill Lehr</center>

## Minimum Requirements

Adios 3 requires certain minimum amounts of data for any oil to be included in the library

Crude Oil:
<ul>
<li>API and/or density at a reference temperature</li>
<li>Viscosity (either kinematic or dynamic) at a reference temperature</li>
</ul>

Refined Product or 'Other' oil:
<ul>
<li>API and/or density at a reference temperature</li>
<li>Viscosity (either kinematic or dynamic) at a reference temperature</li>
<li>At least three distillation cuts giving mass or volume fraction at boiling(bubble) point</li>
</ul>

Unlike Adios 2, Adios 3 stores both structural and distillation fractional components <b><u>(see figure 1)</u></b>.
Certain properties are valid for the whole oil while others may be defined for each structural fraction and still others vary for both the distillation and chemical structural fraction.

The SQL oil database needs a complete set of oil data.  If measured data exists, it takes priority over estimated values.  Many of the more complex estimation formulas are based upon Adios 2 calculations and/or the following reference:

<b><i>Characterization and Properties of Petroleum Fractions</b></i><br>
<b>Author:</b> Dr. M. R. Riazi, Professor of Chemical Engineering, Kuwait University<br>
<b>Published:</b> 2005<br>
<b>Publisher:</b> American Society for Testing and Materials (ASTM), International<br>
<b>Stock No.:</b> MNL50<br>

This will be abbreviated further in this document as CPPF.

## Estimation Rules

### 1. Density:

(A) If no density value exists, estimate it from the oil's API using the following equation:

$$
\begin{align}
\rho0_{oil} &= \text{density of the oil at a } T_{ref} \text{of } 288.15^\circ K \,\, (kg/m^3) \cr
&= {141.5 \over 131.5 + API} \cdot 1000 \qquad \qquad \qquad \qquad \qquad \qquad \boldsymbol{(eq. 1)} \cr
\end{align}
$$

(B) If a density measurement $\rho_0$ at some temperature $T_0$ exists, but no API, then (eq. 1) can be inverted to give an API.  But this can only be done after the density value has been adjusted to be a density at $T_{ref} = 288.15^\circ K (15^\circ C)$.  The density conversion formula for different temperatures is:

$$
\begin{align}
\rho_0 &= \text{measured density} \cr
T_0 &= \text{temperature at which density is measured} \cr
k_{\rho T} &= 0.008 K^{-1} \cr
\cr
\rho_{ref} &= \rho_0 \cdot (1 - k_{\rho T} \cdot (T_{ref} - T_0 )) \qquad \qquad \qquad \qquad \qquad \boldsymbol{(eq. 2)} \cr
\end{align}
$$

<i>(Referenced Source: Adios2)</i>

### 2. Kinematic Viscosity:

If kinematic viscosity value does not exist, estimate from dynamic viscosity.

$$
\begin{align}
\eta 0_{oil} &= \text{measured dynamic viscosity } (kg/(m \cdot s)) \cr
\rho 0_{oil} &= \text{measured density } (kg/m^3) \cr
\cr
v0_{oil} &= \text{kinematic viscosity } (m^2/s) \cr
&= {\eta 0_{oil} \over \rho 0_{oil} } \qquad \qquad \qquad \qquad \qquad \qquad \boldsymbol{(eq. 3)} \cr
\end{align}
$$

<center><b>--- Begin JamesM Comments ---</b></center>

The above is correct, but a bit sparse on detail.  Basically we want to take advantage of any and all measured viscosities that are available.  We will of course prefer the kinematic measurements, but we will want to use any dynamic viscosity measurements that are not redundant.  So the basic program flow will be as follows:

<ul>
<li>Collect all existing measured kinematic viscosities and their associated reference temperatures</li>
<li>Collect any dynamic viscosities measured at temperatures not already represented in the kinematic measurements</li>
<li>Convert the dynamic viscosities into kinematic viscosities using (eq. 3)</li>
<li>Combine our kinematic and converted dynamic viscosities into one aggregate list</li>
</ul>

<center><b>--- End JamesM Comments ---</b></center>

### 3. Oil-Water Surface Tension:

If the imported oil record does not contain a value for surface tension, then we will estimate it using the following:

$$
\begin{align}
\sigma_{o-w} &= \text{oil/water surface tension at } 288.15^\circ K \,\, (N/m) \cr
&= 0.001 \cdot (39 - 0.2571 \cdot API) \qquad \qquad \qquad \qquad \boldsymbol{(eq. 4)} \cr
\end{align}
$$

<i>
Reference: Baker, O. and W. Swerdloff (1956), Calculation of Surface Tensions - Finding the Surface Tension of Hydrocarbon Liquids, Oil Gas J. (2 January 1956) pp. 125
</i>

### 4. Pour Point:

If the imported oil record contains a pour point property then we will simply use it when building the final oil record.<br>
Otherwise, if we have measured molecular weights for the distillation fractions (unusual) then use method <b>(A)</b>.<br>
Otherwise, use method <b>(B)</b>

<b>(A)</b> If molecular weight and mass fractions are given for all the oil fractions $j = 1 \dots jMAX$, then an average molecular weight for the whole oil $\bar M_w$ can be estimated as:

$$
\begin{align}
N &= \text{number of distillation cuts} \cr
jMAX &= 2 (N + 1) \cr
M_{w,j} &= \text{molecular weight of component j} (kg/kmole)\cr
fmass_j &= \text{mass fraction of component j} \cr
\cr
\bar M_w &= \sum_1^{jMAX} M_{w,j} \cdot fmass_j \qquad \qquad \qquad \qquad \boldsymbol{(eq. 5)} \cr
\end{align}
$$

<i>(Note: The calculation for $jMAX$ may seem counterintuitive. It simply states that we sum over all the SARA fractions, each distillation cut represents 1 saturate and 1 aromatic fraction, and that resins and asphaltenes do not have distillation cut data.  So for $N$ distillation cuts, we would calculate $2 N + 2 \rightarrow 2(N + 1)$)</i>

Define $SG = \rho_{oil} / 1000 \cdot kg$ as specific gravity

Then, using CPPF eq. 3.119, our pour point temperature is calculated as:

$$
\begin{align}
T_{API} &= \text{reference temperature for oil kinematic viscosity} \cr
&= 311^\circ K \cr
\cr
T_{pp} &= 130.47 SG^{2.97} \cdot \bar M_w^{0.61235 - 0.47357 SG} \cdot v_{oil}^{0.31 - 0.3283 SG} \cdot T_{API} \qquad \qquad \boldsymbol{(eq. 6)} \cr
\end{align}
$$

<b>(B)</b> Pour point is estimated by reversing the viscosity-temperature correction in Adios2 and assuming that, at the pour point, viscosity is equal to 1 million centistokes.

$$
\begin{align}
c_{v1} &= 5000 K \cr
\cr
T_{pp} &= { c_{v1} \cdot T_{ref} \over c_{v1} - T_{ref} ln(v_{ref}) } \qquad \qquad \qquad \qquad \boldsymbol{(eq. 7)} \cr
\end{align}
$$

<center><b>--- Begin JamesM Comments ---</b></center>

Exactly which $(T_{ref}, v_{ref})$ measurement should we use if multiple viscosity measurements exist?  Since we are calculating pour point, which I understand to mean the lowest temperature for which a finite viscosity exists, I believe we should use the lowest measured temperature and its associated viscosity.

Does the viscosity-temperature correction formula in Adios2 define what $c_{v1}$ is?

<center><b>--- End JamesM Comments ---</b></center>

### 5. Flash Point:

If measured distillation cut data exists, use method <b>(A)</b>.<br>
Otherwise, use method <b>(B)</b>.

<b>(A)</b> Flash point can be estimated from the first pseudo-component cut:

$$
\begin{align}
T_{cut1} &= \text{the boiling point of the first pseudo-component cut } (^\circ K) \cr
\cr
T_{flsh} &= 117 + 0.69 \cdot T_{cut1} \qquad \qquad \qquad \qquad \boldsymbol{(eq. 8)} \cr
\end{align}
$$

<i>
Reference: Chang A., K. Pashakanti, and Y. Liu (2012), Integrated Process Modeling and Optimization, Wiley Verlag.
</i>

<b>(B)</b> Flash point can be estimated from the API value:

$$
\begin{align}
T_{flsh} &= 457 - 3.34 \cdot API \qquad \qquad \qquad \qquad \boldsymbol{(eq. 9)} \cr
\end{align}
$$

### 6. Maximum water fraction of emulsion:

This quantity will be set after the emulsification approach in Adios3 is finalized.  It will vary depending upon the emulsion stability.  For now the following rule will be applied:

If our substance is a crude oil, then $f_{w max} = 0.9$<br>
If our substance is a refined product, then $f_{w max} = 0$

### 7. Bullwinkle (time):

Adios3 needs to know when to initiate emulsification.  The Adios2 development team called this term Bullwinkle.  SINTEF has measured this parameter for some oils, and the new, not yet completed, analysis of emulsification may provide formulas for Bullwinkle.  Bullwinkle may be either a time value (i.e. time delay after which the emulsification formulas are turned on) or a fraction of the oil that needs to evaporate or dissolve before emulsification can start.

Bullwinkle(time) is undefined unless the user explicitly sets a value.  Then it overrides Bullwinkle(fraction) as the determining parameter for the onset of emulsification.

### 8. Bullwinkle (fraction):

<i>
Reference: Adios2
</i>

Bullwinkle is the mass fraction that must evaporate or dissolve before stable emulsification can begin.  This formula will change when we complete the emulsification module.

If $f_{asph} > 0$, then we use method <b>(A)</b><br>
Otherwise, we use method <b>(B)</b><br>

<b>(A)</b> $f_{bull}$ can be calculated from $f_{asph}$:
$$
\begin{align}
f_{bull} &= 0.32 - 3.59 \cdot f_{asph} \qquad \qquad \qquad \boldsymbol{(eq. 10)} \cr
\end{align}
$$

<b>(B)</b> $f_{bull}$ can be calculated from the oil's API:
$$
\begin{align}
f_{bull} &= 0.5762 \cdot log10(API) - 0.6353 \qquad \qquad \boldsymbol{(eq. 11)} \cr
\end{align}
$$

A result of $f_{bull} < 0$ or $f_{bull} > 1$ indicates no emulsification.

### 9. Adhesion <i>(not currently used by model)</i>:

If the imported oil record contains a valid value for adhesion, we will use that value when building our oil.<br>
Otherwise, $Adh_{oil} = 0.035 kg/m^2$

<i>
Reference: ESTC data
</i>

### 10. Sulphur Mass Fraction <i>(not currently used by model)</i>:

If the imported oil record contains a valid value for sulphur mass fraction, we will use that value when building our oil.<br>
Otherwise, $f_{sulf} = 0$

### 11. Solubility:

If the imported oil record contains a valid value for solubility, we will use that value when building our oil.<br>
Otherwise, $c_{solu} = 0 \, kg/m^3$

### 12. Distillation Cut Boiling Point:

<i>
<b>Note:</b> This and the following properties involve quantities that refer to only certain hydrocarbon groups in the oil.  Adios divides the oil up into $N$ distillation cuts and 4 structure groupings. Two of the structure groupings, resins and asphaltenes, are considered inert with regard to evaporation, biodegradation, and dissolution.  The other two groups, saturates and aromatics, do participate at different rates in these processes.
</i>

<i>
<b>Note:</b> If the culled library data for a particular cut is given at a reduced temperature, this cut and any remaining cuts are considered non-volatile.  The boiling point can be set to an arbitrarily high value.
</i>

If no distillation data exists, then we can estimate the distillation data from the oil's API:

First we estimate our lower and upper temperature bounds:

$$
\begin{align}
T_0 &= \text{the lower temperature boundary} \cr
&= 457 - 3.34 \cdot API \qquad \qquad \qquad \qquad \boldsymbol{(eq. 12)} \cr
T_G &= \text{the upper temperature boundary} \cr
&= 1357 - 247.7 \cdot ln(API) \qquad \qquad \qquad \boldsymbol{(eq. 13)} \cr
\end{align}
$$

Next, we would like to generate a set of $N$ temperatures associated with our cuts $T_i$, where $1 \leq i \leq N$.  We will use a default of 5 cuts for this.

$$
\begin{align}
N &= 5 \cr
T_i &= T_0 + T_G \cdot {(i - 1) \over N} \qquad \qquad \qquad \boldsymbol{(eq. 14)} \cr
\end{align}
$$

<i>
Reference: Adios2 and Jones R. (1997), A Simplified Pseudo-component Oil Evaporation Model, Proceedings of the 20th Arctic and Marine Oil Spill Program (AMOP), Vancouver, CA, Vol. 1, pp. 43-62
</i>

<center><b>--- Begin JamesM Comments ---</b></center>

I think we need to clarify $i$ a bit further so there is no confusion here.  $i$ will be used as an index to generate N temperatures.  So $i$ needs to be an integer list of length $N$.  Stating the above equation more precisely along these terms, we might express the following:

$$
\begin{align}
T_i &= T_0 + T_G \cdot {i - 1 \over N} \quad \{ i \in \Bbb Z \,\,|\,\, 1 \leq i \leq N \}\cr
\end{align}
$$

But as a programmer, I like to index my lists starting at zero.  So "I" would state the equation as follows:

$$
\begin{align}
T_i &= T_0 + T_G \cdot {i \over N} \quad \{ i \in \Bbb Z \,\,|\,\, 0 \leq i < N \}\cr
\end{align}
$$

<center><b>--- End JamesM Comments ---</b></center>

### 13. Initial Component Molecular Weight <i>(required for each distillation cut)</i>:

<i>
Reference: CPPF eq. 2.48 and table 2.6
</i>

The saturate and aromatic component molecular weights can be calculated using the distillation cut temperatures as follows:

$$
\begin{align}
M_{w,sat,i} &= \left( 49.677 \cdot \left[ 6.98291 - ln(1070 - T_i) \right] \right)^{3/2}
\qquad \qquad \qquad \boldsymbol{(eq. 15)} \cr
M_{w,arom,i} &= \left( 44.504 \cdot \left[ 6.911 - ln(1015 - T_i) \right] \right)^{3/2}
\qquad \qquad \qquad \boldsymbol{(eq. 16)} \cr
\end{align}
$$

### 14. Initial Resin and Asphaltene Fractions and Densities:

Resins and asphaltenes are two of the four structural divisions that separate the various hydrocarbons in the oil (the other two are saturates and aromatics).  Resins and asphaltene fractional groupings are NOT broken down further by distillation cuts.

<i>
Reference: Fingas empirical formulas that are based upon analysis of ESTC oil properties database.
</i>

If the imported oil record contains valid values for resins and asphaltenes, we will use those values when building our oil.<br>
Otherwise, we will need to estimate them.

First we define some values $A$ and $B$ that will be used for our component fraction formulas:

$$
\begin{align}
\rho 0_{oil} &= \text{oil aggregate density at } 288.15^\circ K \text{  (from eqs. 1,2)} \cr
v0_{oil} &= \text{oil kinematic viscosity at } 288.15^\circ K (source ???)\cr
\cr
A &= 10 \cdot exp(0.001 \cdot \rho 0_{oil}) \qquad \qquad \qquad \qquad \boldsymbol{(eq. 17)} \cr
B &= 10 \cdot ln(1000 \cdot \rho 0_{oil} \cdot v0_{oil}) \qquad \qquad \qquad \boldsymbol{(eq. 18)}\cr
\end{align}
$$

Then our component fraction formulas:

$$
\begin{align}
f_{res} &= 0.033 \cdot A + 0.00087 \cdot B - 0.74,
\qquad \text{if } f_{res} < 0 \text{ then } f_{res} = 0
\qquad \qquad \qquad \boldsymbol{(eq. 19)} \cr
f_{asph} &= 0.000014 \cdot A^3 + 0.000004 \cdot B^2 - 0.18,
\qquad \text{if } f_{asph} < 0 \text{ then } f_{asph} = 0
\qquad \qquad \boldsymbol{(eq. 20)} \cr
\end{align}
$$

<center><b>--- Begin JamesM Comments ---</b></center>

The above formulas describe a simple limit in that the resulting fractions must be greater than zero.  Stating this in simpler mathematical terms:

$$
\begin{align}
f_{res} &= (0.033 \cdot A + 0.00087 \cdot B - 0.74)_{\bot 0} \cr
f_{asph} &= (0.000014 \cdot A^3 + 0.000004 \cdot B^2 - 0.18)_{\bot 0} \cr
\end{align}
$$

I believe that this is not enough to protect us from producing mass fractions that are inconsistent.  If we do not also have an upper limit, then we could end up with a fraction that is greater than 1.0, or a group of fractions that sum to a value greater than 1.0.<br>
In this document, the resin and asphaltene fractions are described first, which implies that we will calculate them before calculating the saturate and aromatic fractions.  This is fine, but the order of calculation should probably be specified.  If we can assume the order of calculation is to first find the resin fraction, and then the asphaltene fraction, then the above formulas including upper and lower limits will look like this:

$$
\begin{align}
f_{res} &= (0.033 \cdot A + 0.00087 \cdot B - 0.74)_{\bot 0}^{\top 1} \cr
f_{asph} &= (0.000014 \cdot A^3 + 0.000004 \cdot B^2 - 0.18)_{\bot 0}^{\top (1 - f_{res})} \cr
\end{align}
$$

<center><b>--- End JamesM Comments ---</b></center>

### 15. Distillation Cut Mass Fraction:

The distillation cuts are a series of temperatures and mass fractions.  And each temperature $T_i$ is associated with a fractional value $fevap_i$ representing the portion of substance that was evaporated at that temperature. 

For any temperature below about $530^\circ K$, we can assume that the portion of substance that was evaporated contains a mix of saturates and aromatics, and that no resins or asphaltenes have been released.  This mix of saturates and asphaltenes, the quantity that was evaporated at a distillation cut temperature, is what we would like to consider to be our distillation cut mass fractions.

<b>(A)</b> If the following prerequisites are true:

- The culled oil library contains summed (both saturate and aromatic combined) mass fractions or weight %, $fevap_i$, for the distillation cuts.
- At least two distillation cuts exist for which:
  - $T_i < 530^\circ K$
  - $0 \leq fevap_i \leq (1 - (f_{res} + f_{asph}))$

Then we will use the distillation cut fractional masses that are supplied to generate $fmass_i$.

The distillation cut mass fractions that we get in the imported oil record are cumulative in nature, so it is necessary to collect the differences in the distillation cut mass fractions when generating $fmass_i$

$$
\begin{align}
fmass_i &= {d \over di} \left( fevap_i \right)
\qquad \{ i \in \Bbb Z \,\,|\,\, 0 \leq i \leq N \}
\qquad \qquad \boldsymbol{(eq. 21)} \cr
\end{align}
$$

It is important that the component mass fractions sum up to 1.0, representing the total mass for the substance.  So the following equation needs to be true.

$$
\begin{align}
1 &= f_{res} + f_{asph} + \sum_{i=0}^{N-1} fmass_i \cr
\end{align}
$$

(<b>Note:</b> If the fractions do not add up to 1.0, we may need to scale them. Exactly how is unclear.  Which ones will we scale, the inert ones or the volatile ones (or both)?  Do we scale evenly?)

<b>(B)</b> If distillation cut temperatures were generated by approximation (eq. 18), then it can be assumed that there were no measured mass fractions.  For this we will evenly distribute our fractional masses such that for all distillation cuts:

$$
\begin{align}
fmass_i &= {(1 - (f_{res} + f_{asph})) \over N}
\qquad \qquad \boldsymbol{(eq. 22)} \cr
\end{align}
$$

### 16. Component Density <i>(required for each distillation cut)</i>:

<i>
Reference: CPPF eq. 2.13 and table 9.6
</i>

Initial density estimate for both resin and asphaltene fractional componens is set to:

$$
\begin{align}
\rho_{res} &= \rho_{asph} = 1100 \, kg/m^3 \cr
\end{align}
$$

Then we define Watson characterization factors for saturates and aromatics:

$$
\begin{align}
K_{arom,w} &= \text{Watson characterization factor for aromatics} \cr
&= 10 \cr
K_{sat,w} &= \text{Watson characterization factor for saturates} \cr
&= 12 \cr
\end{align}
$$

Then we apply the appropriate Watson characterization to calculate an initial trial estimate for each density component:

$$
\begin{align}
\rho try_{arom,i} &= 1000 \cdot {\root 3 \of T_i \over K_{arom,w} } \cr
\rho try_{sat,i} &= 1000 \cdot {\root 3 \of T_i \over K_{sat,w} } \cr
\end{align}
$$

This should be a reasonable estimate.  However, the average density (fractionally weighted average of the cut densities) must match the measured value or the value from approximation (eqs. 1, 2).<br>
To find the scaling factor between our densities and the aggregate density, we do the following:

$$
\begin{align}
j &= \text{index representing all oil components } \{ j \in \Bbb Z \} \cr
jMAX &= \text{index of the last oil component} \cr
&= 2N + 2 \cr
\cr
Cf_{dens} &= { \rho 0_{oil} \over \sum_{j=1}^{jMAX} fmass0_j \cdot \rho try_j} \cr
\end{align}
$$

Then we evenly apply this scaling factor to our trial densities to come up with densities that are consistent with the aggregate density.

$$
\begin{align}
\rho_j = Cf_{dens} \cdot \rho try_j
\end{align}
$$

### 17. Component Specific Gravity:

Specific Gravity of a substance is the ratio of density of the substance to the density of water at a specified temperature.<br>

<i>
Reference: https://en.wikipedia.org/wiki/Specific_gravity
</i>

For simplicity, we will make the assumption that our water density is $1000 \, kg/m^3$.  So our estimation of the specific gravity of our oil components will be:

$$
\begin{align}
SG_j &= \rho_j / 1000 \cr
\end{align}
$$

### 17-18. Initial Saturate and Aromatic Mass Fractions <i>(required for each distillation cut)</i>:

Once we have either measured or estimated distillation cut mass fractions available to us, we can then compute our saturate mass fraction using the specific gravity and molecular weight of the saturate component:

$$
\begin{align}
f_{sat,i} &= (fmass_i \cdot (2.24 - 1.98 \cdot SG_{sat,i} - 0.009 \cdot M_{w,sat,i}))_{\bot 0}^{\top fmass_i} \cr
\end{align}
$$

<i>
Reference: CPPF eqs. 3.77 and 3.78
</i>

After our saturate mass fraction has been computed, we compute our aromatic mass fraction simply as:

$$
\begin{align}
f_{arom,i} &= fmass_i - f_{sat,i} \cr
\end{align}
$$

<center><b>--- Begin JamesM Comments ---</b></center>

But the bigger problem is that our procedure <b>(A)</b> is using a formula that requires two inputs that have not been defined yet.
- Component Specific Gravity: This might be a bit trickier.  Discussion to follow.

We really don't have a section describing $SG_{sat,i}$.  Section 4 (Pour Point) describes oil specific gravity as $SG = \rho_{oil}/1000$, which seems like a reasonable calculation of the density ratio to water based on some quick research.  Specific Gravity of a substance is the ratio of density of the substance to the density of water at $4^\circ C$.  We could use a similar function for this like:

$$
\begin{align}
SG_{sat,i} &= \rho_{sat,i} / 1000 \cr
\end{align}
$$

We have a section for computing component densities, which we could similarly put befor this section.  But we may have a dependency problem.

The component density calculations in this document are dependent on the following:
- Watson characterization factors (constants, so we should be good here)
- Distillation cut temperatures (already calculated, so we should be good here)
- Average oil density (already calculated, so we should be good here)
- distillation cut fractional mass (we have either the data or estimated values, so we should be good here)


<center><b>--- End JamesM Comments ---</b></center>